# Exceptional Days and Periods


The dataset from zugfinder.net, which is the base of our model, was taken between 15.21.2019 and 31.05.2022. During this period, in addition to the usual holidays and holidays, there was also a significant impact on passenger behaviour and ultimately on the timetable due to the Covid 19 pandemic.
During this period, in addition to the usual national holidays and local holidays, there was also a significant impact on passenger behaviour and ultimately on the timetable due to the Covid 19 pandemic.
We have researched all these days and more and made them available to our model so that they can be interpreted accordingly by the model. 

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import re
import datetime as dt

# Train cancellations due to Covid 19 lockdowns in 2020 and 2021

In [3]:
df = pd.read_csv("../data/Corona.csv")

In [4]:
# Forming the data set, rename and reorga of the columns
df["3"]=df["3"].fillna(9999)
df["3"]=df["3"].astype(int)
df["3"]=df["3"].astype(str)
df["2020"]=df["2020"].fillna(9999)
df["2020"]=df["2020"].astype(int)
df=pd.DataFrame(np.vstack([df.columns, df]))
df["train"]=df[2]+" "+(df[3])
df = df.drop(columns=[2,3,7])
df.columns=['id','year','start','end','issue','train']

In [4]:
df.tail(2)

,id,year,start,end,issue,train
629,20200109596,2020,München,Paris Est,Corona: Zug entfällt 20.3.-23.6.,TGV 9596
630,20210109596,2021,München,Paris Est,Corona: Zug beginnt bis 27.6. in Stuttgart Hbf,TGV 9596


### Keep only the trains which where cancelled ("entfällt")
Using Regex101 for the logic.
Create a new column for start and end date of those trains


In [14]:
bdate_list = []
edate_list = []

for index, row in df.iterrows():
    word = "entfällt"
    try:
        if word in str(row["issue"]):
            pattern= "\d+" 
            result = re.findall(pattern, row["issue"])
            if int(result[0]) > 31:
                bdate=None
                edate=None
                pass
                
            if int(result[1]) > 12:
                bdate=None
                edate=None
                pass
            startday=int(result[0])
            startmonth=int(result[1])
            endday=int(result[2])
            endmonth=int(result[3])
            year=int(row["year"])
            
            bdate=dt.datetime(year,startmonth,startday)
            edate=dt.datetime(year,endmonth,endday,)
            
        else:
            bdate=None
            edate=None
    except:
        bdate=None
        edate=None
        
    bdate_list.append(bdate)
    edate_list.append(edate)

In [19]:
df['start_date'] = bdate_list
df['end_date'] = edate_list
df

,id,year,start,end,issue,train,start_date,end_date
0,20200100003,2020,Karlsruhe,Zürich HB,Corona: Zug endet 19.3.-24.5. in Basel Bad Bf,ICE 3,NaT,NaT
1,20200100004,2020,Zürich HB,Frankfurt(Main),Corona: Zug beginnt 19.3.-24.5. in Basel Bad Bf,ICE 4,NaT,NaT
2,20200100005,2020,Frankfurt(Main),Basel SBB,Corona: Zug endet 19.3.-24.5. in Basel Bad Bf,ICE 5,NaT,NaT
3,20200100006,2020,Interlaken Ost,Hamburg-Altona,Corona: Zug endet 19.3.-24.5. in Basel SBB,EC 6,NaT,NaT
4,20200100007,2020,Hamburg-Altona,Interlaken Ost,Corona: Zug beginnt 19.3.-25.5. in Basel SBB,EC 7,NaT,NaT
...,...,...,...,...,...,...,...,...
626,20200109586,2020,Frankfurt(Main),Paris Est,Corona: Zug entfällt 20.3.-10.5.,ICE 9586,2020-03-20,2020-05-10
627,20200109592,2020,Stuttgart,Paris Est,Corona: Zug entfällt 20.3.-23.6.,TGV 9592,2020-03-20,2020-06-23
628,20200109593,2020,Paris Est,Stuttgart,Corona: Zug entfällt 20.3.-23.6.,TGV 9593,2020-03-20,2020-06-23
629,20200109596,2020,München,Paris Est,Corona: Zug entfällt 20.3.-23.6.,TGV 9596,2020-03-20,2020-06-23


In [9]:
df.dtypes # to check the different data types

id                    object
year                  object
start                 object
end                   object
issue                 object
train                 object
start_date    datetime64[ns]
end_date      datetime64[ns]
dtype: object

In [10]:
pd.set_option('display.max_colwidth', None)  # to display all content 
df['issue']

0        Corona: Zug endet 19.3.-24.5. in Basel Bad Bf
1      Corona: Zug beginnt 19.3.-24.5. in Basel Bad Bf
2        Corona: Zug endet 19.3.-24.5. in Basel Bad Bf
3           Corona: Zug endet 19.3.-24.5. in Basel SBB
4         Corona: Zug beginnt 19.3.-25.5. in Basel SBB
                            ...                       
626                   Corona: Zug entfällt 20.3.-10.5.
627                   Corona: Zug entfällt 20.3.-23.6.
628                   Corona: Zug entfällt 20.3.-23.6.
629                   Corona: Zug entfällt 20.3.-23.6.
630     Corona: Zug beginnt bis 27.6. in Stuttgart Hbf
Name: issue, Length: 631, dtype: object

In [20]:
df.head(40)['issue']     # to check in which row the relevant content starts (row 0 - 4 (above) is not relevant) 

0                                                         Corona: Zug endet 19.3.-24.5. in Basel Bad Bf
1                                                       Corona: Zug beginnt 19.3.-24.5. in Basel Bad Bf
2                                                         Corona: Zug endet 19.3.-24.5. in Basel Bad Bf
3                                                            Corona: Zug endet 19.3.-24.5. in Basel SBB
4                                                          Corona: Zug beginnt 19.3.-25.5. in Basel SBB
5                                                            Corona: Zug endet 19.3.-24.5. in Basel SBB
6                                                          Corona: Zug beginnt 19.3.-25.5. in Basel SBB
7          Corona: Zug entfällt 19.3.-24.5. (25.-31.5. Entfall zwischen Köln Hbf und Bruxelles/Brussel)
8                                                                       Corona: Zug entfällt 19.3.-1.6.
9                                                               

### A list of the most frequent days, for the start and end of the train cancellation 

In [17]:
pd.DataFrame(df['end_date'].value_counts()).head(25) 

,end_date
2020-06-13,43
2020-04-05,39
2020-06-23,20
2020-06-14,17
2020-06-07,13
2020-06-12,13
2021-03-27,12
2020-05-31,12
2020-06-22,12
2020-05-28,11


In [ ]:
#data export


Annex of the frequency of the occurring days for "not applicable" we have decided on two periods. From 15.03.2020 to 23.06.2020 and from 10.01.2021 to 24.05.2021. 
By comparing our data with the Google Mobility Report for 2020 and the Covid 19 Mobility Project for 2020 and 2021, we were also able to verify our data. 

# Holidays


In Bayern and NRW December 2019 - January 2023

In [2]:
df = pd.read_csv("../data/holidays_org.csv")

In [3]:
df.head(10)

,nat. holidays since Dec2019,related brueckentage 2019,bayern,start_sholiday_by,end_sholiday_by,start_sholiday_nrw,end_sholiday_nrw,st_carnival,en_carnival
0,2019-12-24,2019-12-27,2020-01-06,2019-12-21,2020-01-06,2019-12-21,2020-01-06,2020-02-20,2020-02-25
1,2019-12-25,2020-05-22,2020-08-08,2020-02-22,2020-03-01,2020-04-04,2020-04-19,2021-02-11,2021-02-16
2,2019-12-26,2020-06-12,2020-08-15,2020-04-04,2020-04-19,2020-05-30,2020-06-02,2022-02-24,2022-03-01
3,2020-01-01,2021-05-14,2021-01-06,2020-06-01,2020-06-14,2020-06-27,2020-08-11,NaN,NaN
4,2020-04-10,2021-06-04,2022-01-06,2020-07-25,2020-09-07,2020-10-10,2020-10-25,NaN,NaN
5,2020-04-12,2022-05-27,2022-08-08,2020-10-31,2020-11-08,2020-12-19,2021-01-06,NaN,NaN
6,2020-05-01,2022-06-17,2022-08-15,2020-12-19,2021-01-10,2021-03-27,2021-04-11,NaN,NaN
7,2020-05-21,NaN,NaN,2021-03-27,2021-04-11,2021-05-22,2021-05-25,NaN,NaN
8,2020-06-01,NaN,NaN,2021-05-22,2021-06-06,2021-07-03,2021-08-17,NaN,NaN
9,2020-06-11,NaN,NaN,2021-07-30,2021-09-13,2021-10-09,2021-10-24,NaN,NaN


In [4]:
# deldays, the days with will be flag or excluded for the model
df_deldays = df['nat. holidays since Dec2019']     
df_deldays=pd.DataFrame(df_deldays)
df_deldays.insert(1, 'holidays', 1)

In [5]:
df_deldays.rename(columns={"nat. holidays since Dec2019": "date"},inplace=True)

In [6]:
df_deldays

,date,holidays
0,2019-12-24,1
1,2019-12-25,1
2,2019-12-26,1
3,2020-01-01,1
4,2020-04-10,1
5,2020-04-12,1
6,2020-05-01,1
7,2020-05-21,1
8,2020-06-01,1
9,2020-06-11,1


In [7]:
# forming dataset/df
df_brudays = df['related brueckentage 2019']     
df_brudays=pd.DataFrame(df_brudays)        ## important to create DataFrame df_brudays
df_brudays.insert(1, 'brudays', 1)
df_brudays.rename(columns={"related brueckentage 2019": "date"},inplace=True)

In [8]:
df_brudays.head(10)

,date,brudays
0,2019-12-27,1
1,2020-05-22,1
2,2020-06-12,1
3,2021-05-14,1
4,2021-06-04,1
5,2022-05-27,1
6,2022-06-17,1
7,NaN,1
8,NaN,1
9,NaN,1


In [9]:
df_deldays.loc[41] = [pd.to_datetime('2019-12-27', format='%Y-%m-%d'), 1]  # adding a row
 # df.index = df.index + 1  # shifting index
 # df = df.sort_index()  # sorting by index

In [10]:
# saving the days to CSV file
df_deldays.to_csv('../data/holidays_deldays.csv', encoding='utf-8', index=False)  

In [11]:
df_deldays

,date,holidays
0,2019-12-24,1
1,2019-12-25,1
2,2019-12-26,1
3,2020-01-01,1
4,2020-04-10,1
5,2020-04-12,1
6,2020-05-01,1
7,2020-05-21,1
8,2020-06-01,1
9,2020-06-11,1


In [12]:
df_bysch=df[df['start_sholiday_by'].notna()]   #to drop the NaNs

In [13]:
df_bysch

,nat. holidays since Dec2019,related brueckentage 2019,bayern,start_sholiday_by,end_sholiday_by,start_sholiday_nrw,end_sholiday_nrw,st_carnival,en_carnival
0,2019-12-24,2019-12-27,2020-01-06,2019-12-21,2020-01-06,2019-12-21,2020-01-06,2020-02-20,2020-02-25
1,2019-12-25,2020-05-22,2020-08-08,2020-02-22,2020-03-01,2020-04-04,2020-04-19,2021-02-11,2021-02-16
2,2019-12-26,2020-06-12,2020-08-15,2020-04-04,2020-04-19,2020-05-30,2020-06-02,2022-02-24,2022-03-01
3,2020-01-01,2021-05-14,2021-01-06,2020-06-01,2020-06-14,2020-06-27,2020-08-11,NaN,NaN
4,2020-04-10,2021-06-04,2022-01-06,2020-07-25,2020-09-07,2020-10-10,2020-10-25,NaN,NaN
5,2020-04-12,2022-05-27,2022-08-08,2020-10-31,2020-11-08,2020-12-19,2021-01-06,NaN,NaN
6,2020-05-01,2022-06-17,2022-08-15,2020-12-19,2021-01-10,2021-03-27,2021-04-11,NaN,NaN
7,2020-05-21,NaN,NaN,2021-03-27,2021-04-11,2021-05-22,2021-05-25,NaN,NaN
8,2020-06-01,NaN,NaN,2021-05-22,2021-06-06,2021-07-03,2021-08-17,NaN,NaN
9,2020-06-11,NaN,NaN,2021-07-30,2021-09-13,2021-10-09,2021-10-24,NaN,NaN


To merge school holidays Bay and NRW and to get all the single days by date


In [14]:
# create a data frame with all dates which are part of the school holidays (include the weekends)
empty=[]
for index,row in df_bysch.iterrows():
    #print(row)
    start=row['start_sholiday_by'] 
# ?print(row['start_sholiday_by'])
    end=row[ 'end_sholiday_by']
    #print(row[ 'end_sholiday_by'])
    empty.append(pd.date_range(start=start, end=end))
empty

[DatetimeIndex(['2019-12-21', '2019-12-22', '2019-12-23', '2019-12-24',
                '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28',
                '2019-12-29', '2019-12-30', '2019-12-31', '2020-01-01',
                '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05',
                '2020-01-06'],
               dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25',
                '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
                '2020-03-01'],
               dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07',
                '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11',
                '2020-04-12', '2020-04-13', '2020-04-14', '2020-04-15',
                '2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19'],
               dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2020-06-01', '2020-06-02', '2020-06-03', '2020-06-

In [15]:
pd.DataFrame(empty[0])

,0
0,2019-12-21
1,2019-12-22
2,2019-12-23
3,2019-12-24
4,2019-12-25
5,2019-12-26
6,2019-12-27
7,2019-12-28
8,2019-12-29
9,2019-12-30


In [16]:
df = pd.Series(empty[0])

In [17]:
df

0    2019-12-21
1    2019-12-22
2    2019-12-23
3    2019-12-24
4    2019-12-25
5    2019-12-26
6    2019-12-27
7    2019-12-28
8    2019-12-29
9    2019-12-30
10   2019-12-31
11   2020-01-01
12   2020-01-02
13   2020-01-03
14   2020-01-04
15   2020-01-05
16   2020-01-06
dtype: datetime64[ns]

In [18]:
empty2 =pd.DataFrame(empty[0])
for item in empty:
    df = pd.Series(item)
    empty2.append(df, ignore_index=True)
empty2


/var/folders/mv/43241nxd07z663ggm5_d6h8h0000gn/T/ipykernel_58105/840812104.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empty2.append(df, ignore_index=True)


,0
0,2019-12-21
1,2019-12-22
2,2019-12-23
3,2019-12-24
4,2019-12-25
5,2019-12-26
6,2019-12-27
7,2019-12-28
8,2019-12-29
9,2019-12-30


In [19]:
pd.DataFrame(np.array(empty).T,dtype=object)

/var/folders/mv/43241nxd07z663ggm5_d6h8h0000gn/T/ipykernel_58105/797563784.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pd.DataFrame(np.array(empty).T,dtype=object)


,0
0,"DatetimeIndex(['2019-12-21', '2019-12-22', '20..."
1,"DatetimeIndex(['2020-02-22', '2020-02-23', '20..."
2,"DatetimeIndex(['2020-04-04', '2020-04-05', '20..."
3,"DatetimeIndex(['2020-06-01', '2020-06-02', '20..."
4,"DatetimeIndex(['2020-07-25', '2020-07-26', '20..."
5,"DatetimeIndex(['2020-10-31', '2020-11-01', '20..."
6,"DatetimeIndex(['2020-12-19', '2020-12-20', '20..."
7,"DatetimeIndex(['2021-03-27', '2021-03-28', '20..."
8,"DatetimeIndex(['2021-05-22', '2021-05-23', '20..."
9,"DatetimeIndex(['2021-07-30', '2021-07-31', '20..."


In [23]:
len(empty)

18

final files added with data Frm Berlin and Brandenburg

see files sholidays_bb.csv (school holidays for Bayern, NRW, Berlin und Brandenburg, plus Carnival(Rheinland)
and file holidays_bru_bb.csv
and file holidays_bb.csv without bridging daysj

#
# Bridge damage in Wiesbaden

On the website 'Datenbank Fernverkehr' we found a data set which shows all the cancelled, rerouted and split trains trains caused by a bridge damage near Wiesbaden which is one route option for trains going between Cologne and Munich.


In [2]:
df = pd.read_csv("../data/Bridge_damage.csv")

In [3]:
df.head(10)

,Datenbank Fernverkehr,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Dieses Archiv bietet umfangreiche Recherchemög...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,deutschenglish»,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sonderabfragen: Geänderte Fernverkehrszüge auf...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Zeigt alle Fernverkehrszüge an, die aufgrund d...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Es wurden 45 Datensätze gefunden.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Zug-ID,Fahrplanjahr,Zuggattung,Zugnummer,Startbahnhof,Zielbahnhof,Grund,NaN,train
6,20210100510,2021,ICE,510,München,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankf...,ansehen,ICE510
7,20210100887,2021,ICE,887,Hamburg-Altona,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankf...,ansehen,ICE887
8,20210101190,2021,ICE,1190,Frankfurt(M)Flughafen Fernbf,Wiesbaden,Zug kommt als ICE 1110 aus München Hbf; Brücke...,ansehen,ICE1190
9,20210101510,2021,ICE,1510,Stuttgart,Wiesbaden,Brückenschaden: Zug endet seit 21.6. in Mainz Hbf,ansehen,ICE1510


In [4]:
# create a useful name for each column
df.columns=['id','year','type','tNo','start','end','damage','del','train']

In [5]:
# to delete non needed rows 
df=df.dropna(axis = 0, subset = "year") 
df=df.dropna(axis = 0, subset = "del")


In [6]:
df.head(10)

,id,year,type,tNo,start,end,damage,del,train
6,20210100510,2021,ICE,510,München,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankf...,ansehen,ICE510
7,20210100887,2021,ICE,887,Hamburg-Altona,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankf...,ansehen,ICE887
8,20210101190,2021,ICE,1190,Frankfurt(M)Flughafen Fernbf,Wiesbaden,Zug kommt als ICE 1110 aus München Hbf; Brücke...,ansehen,ICE1190
9,20210101510,2021,ICE,1510,Stuttgart,Wiesbaden,Brückenschaden: Zug endet seit 21.6. in Mainz Hbf,ansehen,ICE1510
10,20210101554,2021,ICE,1554,Dresden,Wiesbaden,Brückenschaden: Zug endet 18.-26.6. in Frankfu...,ansehen,ICE1554
11,20210101556,2021,ICE,1556,Dresden,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankf...,ansehen,ICE1556
12,20210101558,2021,ICE,1558,Dresden,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankf...,ansehen,ICE1558
13,20210101650,2021,ICE,1650,Dresden,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankf...,ansehen,ICE1650
14,20210101652,2021,ICE,1652,Dresden,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankf...,ansehen,ICE1652
15,20210101654,2021,ICE,1654,Dresden,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankf...,ansehen,ICE1654


In [7]:
# to display all the content of the  the 'damage' column 
pd.set_option('display.max_colwidth', None)  
df['damage']

6     Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flugh Fernbf (16.8.-2.9. und 4.-28.10. in Mainz)
7                                                  Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Hbf
8     Zug kommt als ICE 1110 aus München Hbf; Brückenschaden: Zug endet seit 20.6. in Frankfurt Flugh Fern
9                                                        Brückenschaden: Zug endet seit 21.6. in Mainz Hbf
10        Brückenschaden: Zug endet 18.-26.6. in Frankfurt(M)Flughafen Fernbf, ab 27.6. in Frankfurt(M)Hbf
11                                    Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flughafen Fernbf
12                                    Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flughafen Fernbf
13                                    Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flughafen Fernbf
14                    Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flughafen Fernbf/Frankfurt(M)Hbf
15                                   

In [10]:
df.head(10)

,id,year,type,tNo,start,end,damage,del,train
6,20210100510,2021,ICE,510,München,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flugh Fernbf (16.8.-2.9. und 4.-28.10. in Mainz),ansehen,ICE510
7,20210100887,2021,ICE,887,Hamburg-Altona,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Hbf,ansehen,ICE887
8,20210101190,2021,ICE,1190,Frankfurt(M)Flughafen Fernbf,Wiesbaden,Zug kommt als ICE 1110 aus München Hbf; Brückenschaden: Zug endet seit 20.6. in Frankfurt Flugh Fern,ansehen,ICE1190
9,20210101510,2021,ICE,1510,Stuttgart,Wiesbaden,Brückenschaden: Zug endet seit 21.6. in Mainz Hbf,ansehen,ICE1510
10,20210101554,2021,ICE,1554,Dresden,Wiesbaden,"Brückenschaden: Zug endet 18.-26.6. in Frankfurt(M)Flughafen Fernbf, ab 27.6. in Frankfurt(M)Hbf",ansehen,ICE1554
11,20210101556,2021,ICE,1556,Dresden,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flughafen Fernbf,ansehen,ICE1556
12,20210101558,2021,ICE,1558,Dresden,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flughafen Fernbf,ansehen,ICE1558
13,20210101650,2021,ICE,1650,Dresden,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flughafen Fernbf,ansehen,ICE1650
14,20210101652,2021,ICE,1652,Dresden,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flughafen Fernbf/Frankfurt(M)Hbf,ansehen,ICE1652
15,20210101654,2021,ICE,1654,Dresden,Wiesbaden,Brückenschaden: Zug endet seit 18.6. in Frankfurt(M)Flughafen Fernbf,ansehen,ICE1654


Here it became obvious that the trains which are important for our train routes (Berlin-Cologne-Munich) are not or only indirectly affected. Consequently, we have not considered this data set further for our models.